In [18]:
from sentence_transformers import SentenceTransformer

In [54]:
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [20]:
model_name = 'multi-qa-distilbert-cos-v1'

In [21]:
embedding_model = SentenceTransformer(model_name)

## Q1

In [22]:
user_question = "I just discovered the course. Can I still join it?"

In [23]:
response = embedding_model.encode(user_question)

In [43]:
v = response

In [24]:
len(response)

768

In [25]:
response[0]

0.078222655

## Q2

In [26]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [27]:
type(documents)

list

In [28]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [29]:
documents[0]['course']

'data-engineering-zoomcamp'

In [30]:
def filter_document(documents):
    new_documents = documents
    documents_list = []
    for i in range(len(new_documents)):
        if (new_documents[i]['course'] == 'machine-learning-zoomcamp'):
            documents_list.append(new_documents[i])
    return documents_list
    

In [31]:
new_documents = filter_document(documents)
len(new_documents)

375

In [37]:
def create_embeddings(model, documents):
    embeddings = []
    for doc in tqdm(documents):
        question = doc['question']
        text = doc['text']
        qa_text = f'{question} {text}'
        encoded = model.encode(qa_text)
        embeddings.append(encoded)
    X = np.array(embeddings)
    return X

In [38]:
X = create_embeddings(embedding_model, new_documents)

100%|██████████| 375/375 [01:03<00:00,  5.91it/s]


In [40]:
X.shape

(375, 768)

## Q3

In [44]:
scores = X.dot(v)

In [45]:
scores.argmax()

14

In [46]:
scores[scores.argmax()]

0.6506573

## Q4

In [51]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


In [52]:
search_engine = VectorSearchEngine(documents=new_documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c